In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from core.data.utils import read_all_datasets, read_all_metadata
from core.data.processing.normalization import robust_zscore_normalization_per_dataset

In [2]:
DATA_DIR = '../../db'

dataset, dataset_label = read_all_datasets(DATA_DIR, dropna=False)
metadata = read_all_metadata(DATA_DIR)

In [3]:
y = metadata[metadata.disease.isin(['Ctrl', 'UC']) & metadata.time_of_biopsy.isin([None, 'W0', 'Before'])].disease
dataset_labels = dataset_label.loc[y.index]
X = dataset.loc[y.index]
X = X.dropna(axis=1)

X.shape, y.shape

((934, 14279), (934,))

In [4]:
X_norm = robust_zscore_normalization_per_dataset(X, dataset_label)

In [5]:
from core.utils.genes import entrez_id_to_gene_symbol

X_norm = entrez_id_to_gene_symbol(X_norm)

20 input query terms found no hit:	['10638', '129790', '157247', '158295', '158696', '23285', '26148', '283174', '283588', '284083', '2


In [31]:
from core.stats.tests import test_two_groups


test_data = X_norm.astype(float)
test_data['disease'] = y

test_two_groups(test_data, 'disease')

Test results for disease:
Group counts: 235 | 699
- BACE2: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.9087
- CCL11: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.7703
- GPX8: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.8996
- DAPP1: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.8065
- SAMD9L: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.7356
- PDZK1IP1: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.8228
- IL1B: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.8137
- COL1A2: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.7676
- SLC39A5: [Mann-Whitney U test] p-value = 0.0; mean difference = -0.7068
- C4BPB: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.7185
- DUOX2: [Mann-Whitney U test] p-value = 0.0; mean difference = 1.1104
- UBE2L6: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.8407
- TGFBI: [Mann-Whitney U test] p-value = 0.0; mean difference = 0.784
- CENPV: [Mann-Whitne